<a href="https://colab.research.google.com/github/Baekhyeonjeong/movie-review-/blob/main/%EC%98%81%ED%99%94_%EC%8A%A4%EC%9C%99%EA%B1%B8%EC%A6%88_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [147]:
# konlpy 설치하기
!pip install konlpy

In [148]:
# konlpy를 실행하기
import konlpy

In [149]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [150]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [151]:
## 문서 ID로 실제 파일 불러오기
# 스윙걸즈 (1).csv
## https://drive.google.com/file/d/1da1gcGEnqKJI1o89Gegy6b8T9HSlzQmi/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1da1gcGEnqKJI1o89Gegy6b8T9HSlzQmi'})
rawdata_downloaded.GetContentFile('스윙걸즈 (1).csv')

In [152]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('스윙걸즈 (1).csv', sep=",")

In [153]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,mapl****,일본영화 별로 안좋아하지만 이건 진짜 추천.. 기대안하고봤는데 너무 너무 유쾌하고 ...,10,111,16,2013-09-02 14:23:00,0
1,1,als2****,마지막 연주 3곡은 보는내내 가슴이 짜릿했다!!,10,59,17,2014-01-02 10:42:00,0
2,2,a141****,만화같고 연기도 유치하지만 학창시절 판타지란 이런거 라는걸 보여주는거 같다...,7,53,20,2011-03-09 16:35:00,0
3,3,타우르(wwol****),명작이다 이거뉴ㅠㅠ몇번보고 또봤지만 잼있다!!,10,40,15,2014-06-25 19:04:00,0
4,4,윤(o121****),질리지 않는 영화. 여름마다 생각나는 영화. 늘 새롭고 유쾌하다 빅밴드 하모니도 정...,10,37,14,2014-07-01 00:13:00,0
...,...,...,...,...,...,...,...,...
3905,3905,snic****,오케스트라건 빅밴드건 찰떡같이 잘 어울리는 능청스런 우에노 쭈리의 오바연기!! ㅎㅎ,8,0,7,2019-02-12 23:57:00,0
3906,3906,Shaun(lovi****),15년 전에 나온영화지만 지금까지 내 인생 베스트영화!! 다시 봐도 감동이다..,10,0,7,2019-02-05 01:15:00,0
3907,3907,에버그린(paju****),우에노주리 연기는 실제 본인의 내면에 귀염움이나 장난끼가 없으면 저렇게 나올수 없는...,10,1,8,2019-01-27 13:01:00,0
3908,3908,keer****,2011년도 스무살에 초에 보고 스물후반에다시본 스윙걸즈 시간이 지났음에도 그때와 ...,10,2,9,2018-11-11 21:51:00,0


In [154]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [155]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,mapl****,일본영화 별로 안좋아하지만 이건 진짜 추천.. 기대안하고봤는데 너무 너무 유쾌하고 ...,10,111,16,2013-09-02 14:23:00,0
1,1,als2****,마지막 연주 3곡은 보는내내 가슴이 짜릿했다!!,10,59,17,2014-01-02 10:42:00,0
2,2,a141****,만화같고 연기도 유치하지만 학창시절 판타지란 이런거 라는걸 보여주는거 같다...,7,53,20,2011-03-09 16:35:00,0
3,3,타우르(wwol****),명작이다 이거뉴ㅠㅠ몇번보고 또봤지만 잼있다!!,10,40,15,2014-06-25 19:04:00,0
4,4,윤(o121****),질리지 않는 영화. 여름마다 생각나는 영화. 늘 새롭고 유쾌하다 빅밴드 하모니도 정...,10,37,14,2014-07-01 00:13:00,0
...,...,...,...,...,...,...,...,...
3905,3905,snic****,오케스트라건 빅밴드건 찰떡같이 잘 어울리는 능청스런 우에노 쭈리의 오바연기!! ㅎㅎ,8,0,7,2019-02-12 23:57:00,0
3906,3906,Shaun(lovi****),15년 전에 나온영화지만 지금까지 내 인생 베스트영화!! 다시 봐도 감동이다..,10,0,7,2019-02-05 01:15:00,0
3907,3907,에버그린(paju****),우에노주리 연기는 실제 본인의 내면에 귀염움이나 장난끼가 없으면 저렇게 나올수 없는...,10,1,8,2019-01-27 13:01:00,0
3908,3908,keer****,2011년도 스무살에 초에 보고 스물후반에다시본 스윙걸즈 시간이 지났음에도 그때와 ...,10,2,9,2018-11-11 21:51:00,0


# 형태소 분석

In [156]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [157]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [158]:
# "형태소" 변수의 저장 내용 확인
형태소

[('내', 'Noun'),
 ('가', 'Josa'),
 ('본', 'Verb'),
 ('일', 'Modifier'),
 ('본', 'Modifier'),
 ('영화', 'Noun'),
 ('중', 'Suffix'),
 ('에', 'Josa'),
 ('제일', 'Noun'),
 ('재밌음', 'Adjective'),
 ('.', 'Punctuation')]

In [160]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,내,Noun,10,1
1,가,Josa,10,1
2,본,Verb,10,1
3,일,Modifier,10,1
4,본,Modifier,10,1
5,영화,Noun,10,1
6,중,Suffix,10,1
7,에,Josa,10,1
8,제일,Noun,10,1
9,재밌음,Adjective,10,1


In [161]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[        0            1  type  count
 0      일본         Noun    10      1
 1      영화         Noun    10      1
 2      별로         Noun    10      1
 3       안   VerbPrefix    10      1
 4   좋아하지만    Adjective    10      1
 5      이건         Noun    10      1
 6      진짜         Noun    10      1
 7      추천         Noun    10      1
 8      ..  Punctuation    10      1
 9       기     Modifier    10      1
 10     대안         Noun    10      1
 11     하고         Josa    10      1
 12    봤는데         Verb    10      1
 13     너무       Adverb    10      1
 14     너무       Adverb    10      1
 15   유쾌하고    Adjective    10      1
 16    재밌게    Adjective    10      1
 17     봤음         Verb    10      1
 18      .  Punctuation    10      1
 19   발랄하고    Adjective    10      1
 20    명랑한    Adjective    10      1
 21    사춘기         Noun    10      1
 22    여고생         Noun    10      1
 23      들       Suffix    10      1
 24      의         Josa    10      1
 25     모습         Noun    10      1
 

In [162]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [163]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,일본,Noun,10,1
1,영화,Noun,10,1
2,별로,Noun,10,1
3,안,VerbPrefix,10,1
4,좋아하지만,Adjective,10,1
...,...,...,...,...
6,중,Suffix,10,1
7,에,Josa,10,1
8,제일,Noun,10,1
9,재밌음,Adjective,10,1


In [164]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [165]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,일본,Noun,10,1
1,영화,Noun,10,1
2,별로,Noun,10,1
3,안,VerbPrefix,10,1
4,좋아하지만,Adjective,10,1
...,...,...,...,...
6,중,Suffix,10,1
7,에,Josa,10,1
8,제일,Noun,10,1
9,재밌음,Adjective,10,1


In [166]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [167]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소    품사           분류
!      Punctuation  1     3
                    2     1
                    5     1
                    6     2
                    7     9
                         ..
힘들다면   Adjective    10    1
힘들어진다  Adjective    10    1
힘들었고   Adjective    6     1
힘들지    Adjective    10    1
힘없는    Adjective    6     1
Name: 카운트, Length: 10235, dtype: int64

In [168]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [169]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소   품사          분류     
!     Punctuation 1     3
                  2     1
                  5     1
                  6     2
                  7     9
...                   ...
힘들다면  Adjective   10    1
힘들어진다 Adjective   10    1
힘들었고  Adjective   6     1
힘들지   Adjective   10    1
힘없는   Adjective   6     1

[10235 rows x 1 columns]

In [170]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')